본 EDA 파일은 모델에 맞게 샘플링릊 ㅣㄴ행하고자 여러 가지 측면에서 데이터를 보고, 불필요한 데이터 row, col을 걸러내기 위한 작업을 진행합니다

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
path = '/home/siyun/ephemeral/data/'
df = pd.read_csv(path + 'gift_data.csv')

In [3]:
df['product_id'].nunique()

131075

In [8]:
df['category_3'].fillna('unknown', inplace=True)
product_category_counts = df.groupby(['product_id', 'category_1', 'category_2', 'category_3']).size().reset_index(name='counts')

# 각 product_id 내에서 카테고리가 선택될 확률 계산
product_category_counts['total_counts'] = product_category_counts.groupby('product_id')['counts'].transform('sum')
product_category_counts['prob'] = product_category_counts['counts'] / product_category_counts['total_counts']

# 각 product_id에 대한 엔트로피를 계산
# -sigma(pi) * log2(pi)
product_category_counts['personality'] = -product_category_counts['prob'] * np.log2(product_category_counts['prob'])
product_entropy = product_category_counts.groupby('product_id')['personality'].sum().reset_index()

# 매핑
mean_entropy = product_entropy['personality'].mean()
df = df.merge(product_entropy, on='product_id', how='left')
df['personality'].fillna(mean_entropy, inplace=True)

In [ ]:
# df.groupby(['category_1','category_2'])['personality'].hist(alpha=0.5, bins=20)

product_id의 빈도가 최소 2인 것만 반환

각 카테고리1 별 카테고리2의 통계값 확인

In [9]:
def cat_stats(df) : 
    for col1 in df['category_1'].unique():
        # 특정 category_1 값에 해당하는 행들만 선택
        filtered_df1 = df[df['category_1'] == col1]

        # category_1에 대한 price 통계 계산
        mean_price1 = filtered_df1['price'].mean()
        std_price1 = filtered_df1['price'].std()

        mean_review1 = filtered_df1['num_review'].mean()
        std_review1 = filtered_df1['num_review'].std()

        # 결과 출력
        print(f"len : {len(filtered_df1)} || Category_1: {col1} || price_mean :{int(mean_price1)}({std_price1:.3f}) || price_min : {filtered_df1['price'].min()} || Q_0.3: {int(filtered_df1['price'].quantile(0.3))} || Q_0.5: {int(filtered_df1['price'].median())} || Q_0.99: {int(filtered_df1['price'].quantile(0.99))} || price_max : {filtered_df1['price'].max()}")

        print(f"                                        review_mean: {int(mean_review1)}({std_review1:.3f}) || review_min : {filtered_df1['price'].min()} || Q_0.3: {int(np.quantile(std_review1, 0.3))} || Q_0.5: {int(np.median(std_review1))}")
        # print(f"Q 0.3보다 낮은 row의 개수 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)])} 의 비율 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)]) / len(df)}")
        # category_2의 unique 값에 대해 반복
        for col2 in filtered_df1['category_2'].unique():
            # 특정 category_2 값에 해당하는 행들만 선택
            filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

            if len(filtered_df2) <= 1 :
                pass
            else :
                # category_2에 대한 price 통계 계산
                mean_price2 = filtered_df2['price'].mean()
                std_price2 = filtered_df2['price'].std()


                mean_review2 = filtered_df2['num_review'].mean()
                std_review2 = filtered_df2['num_review'].std()
                
                # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 

                # 결과 출력
                print(f"    len : {len(filtered_df2)} || Category_2: {col2} || Category_1: {col1} || price_mean :{int(mean_price2)}({std_price2:.3f}) || price_min : {filtered_df2['price'].min()} || Q_0.3: {int(filtered_df2['price'].quantile(0.3))} || Q_0.5: {int(filtered_df2['price'].median())} || Q_0.99: {int(filtered_df2['price'].quantile(0.99))} || price_max : {filtered_df2['price'].max()}")
                print(f"                                             review_mean: {int(mean_review1)}({std_review1:.3f}) || review_min : {filtered_df2['price'].min()} || Q_0.3: {int(np.quantile(std_review2, 0.3))} || Q_0.5: {int(np.median(std_review2))}")


In [ ]:
cat_stats(df)

filtering

가격
- 비정상적으로 높은 부분을 제거하기 위해(비정상적으로 높다 == 어뷰징의 가능성) std 약 3sigma(97%)의 상한선

리뷰
- 너무 적은 것은 표본으로써 기능을 못한다고 생각함.
- 즉, 인기가 없다 != 독특하다 가 아니라 인기가 없다 == 안사는데 이유가 있다 or 대체제로써 가치가 낮다라고 생각.
- 리뷰는 이에 대한 하나의 척도로써 주관적으로 판단함.


In [13]:
filtered_df = pd.DataFrame()

for col1 in df['category_1'].unique():
    # 특정 category_1 값에 해당하는 행들만 선택
    filtered_df1 = df[df['category_1'] == col1]

    # category_2의 unique 값에 대해 반복
    for col2 in filtered_df1['category_2'].unique():
        # 특정 category_2 값에 해당하는 행들만 선택
        filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

        # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 
        price_q_3 = filtered_df2['price'].quantile(0.3)
        price_q_99 =  filtered_df2['price'].quantile(0.99)
        rev_q_0_50 = filtered_df2['num_review'].quantile(0.5)

        filtered_q3 = filtered_df2[(filtered_df2['price'] > price_q_3) & (filtered_df2['price'] <= price_q_99) & (filtered_df2['num_review'] >= rev_q_0_50)]
        filtered_df = pd.concat([filtered_df, filtered_q3])


In [15]:
filtered_df.shape

(115086, 14)

필터링한 후 확인

In [27]:
cat_stats(filtered_df)

len : 42626 || Category_1: 생일선물 || price_mean :55385(93843.097) || price_min : 1980 || Q_0.3: 23900 || Q_0.5: 31800 || Q_0.99: 489000 || price_max : 2199000
                                        review_mean: 1912(2389.126) || review_min : 1980 || Q_0.3: 2389 || Q_0.5: 2389
    len : 9889 || Category_2: 환갑/칠순선물 || Category_1: 생일선물 || price_mean :78180(107100.488) || price_min : 19810 || Q_0.3: 29900 || Q_0.5: 39900 || Q_0.99: 599000 || price_max : 898000
                                             review_mean: 1912(2389.126) || review_min : 19810 || Q_0.3: 2454 || Q_0.5: 2454
    len : 9314 || Category_2: 4050대 여성 || Category_1: 생일선물 || price_mean :41382(32032.174) || price_min : 14940 || Q_0.3: 23500 || Q_0.5: 30000 || Q_0.99: 178818 || price_max : 236000
                                             review_mean: 1912(2389.126) || review_min : 14940 || Q_0.3: 2466 || Q_0.5: 2466
    len : 6452 || Category_2: 2030대 여성 || Category_1: 생일선물 || price_mean :41429(65189.467) || price_min : 

샘플링할 비율 계산

In [16]:
filtered_df_col = filtered_df[['product_id','category_1','category_2','category_3','rating','num_review','price']]

for col1 in filtered_df_col['category_1'].unique():
    filtered_df1 = filtered_df[filtered_df_col['category_1'] == col1]

    print(f" 카테고리1 [{col1}] ratio : {len(filtered_df1) / len(filtered_df_col)}")

    for col2 in filtered_df1['category_2'].unique():
        filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]
        print(f"\t 카테고리2 [{col2}]의 상대 ratio : {len(filtered_df2) / len(filtered_df1)}")


 카테고리1 [생일선물] ratio : 0.3703838868324557
	 카테고리2 [환갑/칠순선물]의 상대 ratio : 0.23199455731243843
	 카테고리2 [4050대 여성]의 상대 ratio : 0.21850513770937927
	 카테고리2 [2030대 여성]의 상대 ratio : 0.15136301787641346
	 카테고리2 [10대 여성]의 상대 ratio : 0.0803031013935157
	 카테고리2 [2030대 남성]의 상대 ratio : 0.11741660019706282
	 카테고리2 [4050대 남성]의 상대 ratio : 0.14606108947590674
	 카테고리2 [10대 남성]의 상대 ratio : 0.0254070285741097
	 카테고리2 [파티용품]의 상대 ratio : 0.025852765917515133
	 카테고리2 [e쿠폰]의 상대 ratio : 0.001994088115234833
	 카테고리2 [케이크]의 상대 ratio : 0.0011026134284239666
 카테고리1 [퇴직선물] ratio : 0.04211633039639921
	 카테고리2 [화장품]의 상대 ratio : 0.6622653187538684
	 카테고리2 [영양제]의 상대 ratio : 0.21270889209820507
	 카테고리2 [향초_디퓨저]의 상대 ratio : 0.11140911904270683
	 카테고리2 [꽃]의 상대 ratio : 0.013616670105219723
 카테고리1 [취업선물] ratio : 0.07731609405140503
	 카테고리2 [화장품]의 상대 ratio : 0.3596313778377163
	 카테고리2 [데스크테리어]의 상대 ratio : 0.23881771184535852
	 카테고리2 [영양제]의 상대 ratio : 0.3609799955046078
	 카테고리2 [디지털_가전]의 상대 ratio : 0.04057091481231737
 카테고리1 [집

In [17]:
# 카테고리1 별 비율 계산
cat1_ratios = filtered_df['category_1'].value_counts(normalize=True)

# 샘플링할 최종 데이터프레임 초기화
final_sample_df = pd.DataFrame()

# 총 샘플링할 개수
total_samples = 30000

# 카테고리1 별로 루프
for cat1, cat1_ratio in cat1_ratios.items():
    cat1_df = filtered_df[filtered_df['category_1'] == cat1]
    # print(cat1_ratio)

    # 카테고리1의 비율에 따라 할당할 샘플 수 계산 (1 - 카테고리1의 비율)
    cat1_sample_size = int((cat1_ratio) * total_samples)
    # print(cat1_sample_size)

    # 카테고리2 별 비율 계산
    cat2_ratios = cat1_df['category_2'].value_counts(normalize=True)

    # 카테고리2 별로 샘플링
    for cat2, cat2_ratio in cat2_ratios.items():
        cat2_df = cat1_df[cat1_df['category_2'] == cat2]

        # 카테고리2 별 할당할 샘플 수 (카테고리2의 비율 * 카테고리1 내 샘플 수)
        cat2_sample_size = max(3, int(cat2_ratio * cat1_sample_size)) # 적어도 3개는 샘플링

        # 샘플링 후 최종 데이터프레임에 추가
        sampled_df = cat2_df.sample(n=cat2_sample_size, replace=False) # replace=False로 중복 없이 추출
        final_sample_df = pd.concat([final_sample_df, sampled_df])

# 최종 샘플링된 데이터프레임의 길이 조정
# 필요에 따라서는 여기서 추가 조정을 할 수 있습니다. 예를 들어 total_samples보다 많이 샘플링된 경우 조정 필요
if len(final_sample_df) > total_samples:
    final_sample_df = final_sample_df.sample(n=total_samples)
# cat_stats(final_sample_df)

In [18]:
cat_stats(final_sample_df)

len : 11106 || Category_1: 생일선물 || price_mean :55012(92805.814) || price_min : 2000 || Q_0.3: 24000 || Q_0.5: 31800 || Q_0.99: 489000 || price_max : 2199000
                                        review_mean: 1919(2398.381) || review_min : 2000 || Q_0.3: 2398 || Q_0.5: 2398
    len : 2577 || Category_2: 환갑/칠순선물 || Category_1: 생일선물 || price_mean :75506(104133.075) || price_min : 19900 || Q_0.3: 29900 || Q_0.5: 39500 || Q_0.99: 573799 || price_max : 898000
                                             review_mean: 1919(2398.381) || review_min : 19900 || Q_0.3: 2494 || Q_0.5: 2494
    len : 2427 || Category_2: 4050대 여성 || Category_1: 생일선물 || price_mean :40234(29658.663) || price_min : 14950 || Q_0.3: 23800 || Q_0.5: 29900 || Q_0.99: 160000 || price_max : 230000
                                             review_mean: 1919(2398.381) || review_min : 14950 || Q_0.3: 2374 || Q_0.5: 2374
    len : 1681 || Category_2: 2030대 여성 || Category_1: 생일선물 || price_mean :44155(71634.432) || price_min : 

비율 확인

최종적으로 product_id의 빈도가 3이상인 것만 반환

In [19]:
product_id_counts = final_sample_df['product_id'].value_counts()

# 빈도가 2 이상인 product_id 선택
valid_product_ids = product_id_counts[product_id_counts >= 3].index

# 해당 product_id만 포함하는 행을 필터링
final_sample_df_freq_2= final_sample_df[final_sample_df['product_id'].isin(valid_product_ids)]

In [ ]:
cat_stats(final_sample_df_freq_2)

In [22]:
final_sample_df_freq_2.shape

(7869, 14)

In [34]:
final_sample_df_freq_2[(final_sample_df_freq_2['price'] > 70000) & final_sample_df_freq_2['category_2'] == '새차선물' ]

,product_id,product_name,brand,category_1,category_2,category_3,rating,num_review,price,image_url,product_url,personality


In [25]:
print(final_sample_df_freq_2['product_id'].nunique())
# print(final_sample_df_freq_2['item_id'].nunique())

2258


filtered_sampling_df 저장

In [48]:
final_sample_df_freq_2.to_csv('/home/siyun/ephemeral/data/filtered_gift.csv', index=False)

In [10]:
filtered_encoding = pd.get_dummies(final_sample_df_freq_2, columns=['category_1','category_2','category_3'])

In [27]:
data_index = filtered_encoding.copy()
unique_product_ids = np.unique(data_index['product_id']).tolist()
# id2idx: 각 product_id를 연속적인 정수 인덱스로 매핑
id2idx = {product_id: idx for idx, product_id in enumerate(unique_product_ids)}

# idx2id: 연속적인 정수 인덱스를 원래의 product_id로 매핑
idx2id = {idx: product_id for product_id, idx in id2idx.items()}

In [41]:
data_index['mapped_product_id'] = data_index['product_id'].apply(lambda x: id2idx[x])
cols = list(data_index.columns)

In [42]:
features = cols.copy()

# 새로운 feature 리스트 생성
removes = ['product_id','mapped_product_id', 'product_name', 'brand', 'image_url', 'product_url']
for remove in removes : 
    features.remove(remove)
target = ['mapped_product_id']

In [43]:
from sklearn.preprocessing import LabelEncoder

X = data_index[features]
y = data_index[target]

In [48]:
from sklearn.model_selection import train_test_split

# train, test, valid로 나눕시다
train_x,test_x,train_y,test_y = train_test_split(X,y, test_size=0.25, random_state=42, stratify=y)

# x_train,x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

In [49]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier
import wandb

In [39]:
# def train_and_evaluate(X, y, config):
#     # 모델 초기화
#     model = LGBMClassifier(
#         objective='multiclass',  
#         learning_rate=config.learning_rate,
#         num_leaves=config.num_leaves,
#         n_estimators=config.n_estimators,
#         max_depth=config.max_depth,
#         verbose=-1,
#         # n_jobs= 4,
#         device_type = 'cuda' 
#     )

#     # K-폴드 교차 검증 설정
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     # 전체 모델 성능을 위한 리스트 초기화
#     overall_accuracies = []
#     overall_f1_scores = []
#     overall_recalls = []
#     overall_precisions = []
#     overall_auc_scores = []

#     # 레이블별 예측 결과를 저장할 리스트
#     y_pred_list = []

#     for train_index, test_index in skf.split(X, y):
#         X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
#         y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

#         # 레이블별 예측 결과를 저장할 임시 배열
#         model.fit(X_train_fold, y_train_fold)

#         y_pred_fold = model.predict(X_test_fold)

#         # 전체 모델의 성능 계산
#         overall_accuracy = accuracy_score(y_test_fold, y_pred_fold)
#         overall_f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
#         # overall_recall = recall_score(y_test_fold, y_pred_fold.round())
#         # overall_precision = precision_score(y_test_fold, y_pred_fold.round())
#         # overall_auc = roc_auc_score(y_test_fold, y_pred_fold.round())

#         overall_accuracies.append(overall_accuracy)
#         overall_f1_scores.append(overall_f1)
#         # overall_recalls.append(overall_recall)
#         # overall_precisions.append(overall_precision)
#         # overall_auc_scores.append(overall_auc)

#         # 레이블별 예측 결과를 리스트에 추가
#         y_pred_list.append(y_pred_fold)

#     # 평균 전체 모델 성능 로깅
#     wandb.log({
#         "overall_accuracy": np.mean(overall_accuracies),
#         "overall_f1_score": np.mean(overall_f1_scores),
#         # "overall_recall": np.mean(overall_recalls),
#         # "overall_precision": np.mean(overall_precisions),
#         # "overall_auc_score": np.mean(overall_auc_scores)
#     })

# sweep_config = {
#     'method': 'bayes',
#     'metric': {
#         'name': 'accuracy',
#         'goal': 'maximize'   
#     },
#     'parameters': {
#         'learning_rate': {
#             'min': 0.001,
#             'max': 0.1
#         },
#         'num_leaves': {
#             'values': [4,5]
#         },
#         'n_estimators': {
#             'values': [10,20,50]
#         },
#         'max_depth': {
#             'values': [4,8,10]
#         }
#     }
# }

# def sweep_train():
#     # WandB Sweep 실행을 위한 함수
#     with wandb.init() as run:
#         config = wandb.config
#         train_and_evaluate(X, y, config)

# # Sweep 설정을 WandB에 등록하고 Sweep ID 받기
# sweep_id = wandb.sweep(sweep_config, project="filtering")

# # Sweep Agent 시작
# wandb.agent(sweep_id, sweep_train)

# train_and_evaluate(train_x, train_y, params)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: i0rjdepq
Sweep URL: https://wandb.ai/tldbs5026/filtering/sweeps/i0rjdepq


wandb: Agent Starting Run: tktm1u3g with config:
wandb: 	learning_rate: 0.039523765393126895
wandb: 	max_depth: 4
wandb: 	n_estimators: 20
wandb: 	num_leaves: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Ctrl + C detected. Stopping sweep.


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [50]:
# acc,f1 70세팅 : 0.01,4,30,8

params = {
    'learning_rate': 0.01794243998188023,  # 예시 값
    'num_leaves': 4,      # 예시 값
    'n_estimators': 20,    # 예시 값
    'max_depth': 4,         # 예시 값
    'device_type' : 'cuda' 
}

# 전체 데이터에 대한 모델 학습 함수
def train_full_model(X, y, params):
    num_classes = y.nunique()
    model = LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        learning_rate=params['learning_rate'],
        num_leaves=params['num_leaves'],
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        verbose=-1,
        device_type=params['device_type']
    )
    model.fit(X, y)
    return model




모델을 pkl로 저장

In [51]:
model = train_full_model(train_x, train_y, params)

모델 성능 확인

In [42]:
# y_pred = model.predict(test_x)
# train_x,test_x,train_y,test_y

# accuracy = accuracy_score(test_y, y_pred)
# f1 = f1_score(test_y, y_pred, average='macro')
# conf_matrix = confusion_matrix(test_y, y_pred)

# print(accuracy)
# print(f1)
# print(conf_matrix)

In [44]:
# from sklearn.externals import joblib
from lightgbm import LGBMClassifier
import pickle

model_save_path = "/home/siyun/ephemeral/boosting/output/n30000_7469_lgbm_lr_01794243998188023_lea_4_esti_20_dep_4.pkl"  # 변경된 경로
with open(model_save_path, "wb") as file:
    pickle.dump(model, file)

model_save_path

'/home/siyun/ephemeral/boosting/output/n30000_7469_lgbm_lr_01794243998188023_lea_4_esti_20_dep_4.pkl'

In [ ]:
with open(model_save_path, 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
def add_prediction_probabilities(models, X, data, target):
    # 해당 모델에 대한 확률 계산
    prob = model.predict_proba(X)

    # 가장 확률이 높은 클래스의 인덱스 찾기
    highest_prob_index = np.argmax(prob, axis=1)
    # 가장 확률이 높은 클래스에 대한 확률 선택

    # 가장 확률이 높은 클래스의 인덱스를 사용하여 실제 product_id를 매핑
    predicted_product_ids = [idx2id[idx] for idx in highest_prob_index]

    highest_prob = np.max(prob, axis=1)

    # 결과를 데이터에 추가
    data['predicted_product_id'] = predicted_product_ids
    data['highest_probability'] = highest_prob
    
    return data


# train_x에 대한 inference
## pkl로 받은 모델을 사용하기 위해서는 model -> loaded_model
final_prediction = add_prediction_probabilities(model, X, data_index.copy(), 'mapped_product_id')
final_prediction

유사도 계산
- 카테고리 별 계산

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_products(inference, user_product_id, n=10):
    """
    주어진 product_id에 대해 상위 n개의 가장 유사한 predicted_product_id를 찾습니다.
    
    Args:
    - infernce: 예측된 product_id와 그 확률이 포함된 DataFrame.
    - user_product_id: 유저의 product_id.
    - n: 반환할 유사한 product_id의 개수.
    
    Method:
    - 카테고리를 바탕으로 유사도를 계산
    - 예측된 product_id가 해당하는 카테고리의 다른 아이템들간의 유사도를 계산하여 높은 것을 반환. 
    - 추가적으로 다른 scaled data도 고려하여 유사도를 계산

    Returns:
    - 상위 n개의 가장 유사한 predicted_product_id 리스트.
    """
    # print(f"type(user_product_id) : {type(user_product_id)}")
    # print(f"inference['product_id'].dtype : {inference['product_id'].dtype}")
    
    # # 하나의 product_id여도 여러개가 있기 때문에 이를 제거하는 것이 필요
    # print(f"inf안에 product_id가 있는가? : {inference[inference['product_id'] == user_product_id]}")

    unique_products = inference.sort_values(by='highest_probability', ascending=False).drop_duplicates(subset='product_id', keep='first').reset_index(drop=True)

    category_columns = [col for col in unique_products.columns if 'category' in col]
    features = unique_products[category_columns]
    
    # user_product_id가 unique_products에 존재하는지 확인
    if user_product_id in unique_products['product_id'].values:
        # target_idx를 인덱스 값이 아닌 위치로 사용
        target_idx = unique_products.index[unique_products['product_id'] == user_product_id].tolist()[0]
        target_features = features.loc[[target_idx]]
        target_features_2d = target_features.values.reshape(1, -1)
        sim = cosine_similarity(target_features_2d, features)[0]

        sim[target_idx] = -1  # 자기 자신 제외
        similar_indices = sim.argsort()[::-1][:n]
        similar_product_ids = unique_products.iloc[similar_indices]['product_id'].tolist()
        return similar_product_ids
    else:
        # user_product_id에 해당하는 데이터가 없는 경우
        return []


user_product_id를 입력받으면 유사도 계산 -> topk 개의 유사한 아이템 반환

In [ ]:
user_product_id=2864497971
top_similar_products = find_similar_products(final_prediction, user_product_id, n=10)
top_similar_products

확인

In [ ]:
final_prediction[final_prediction['product_id'] == 6423778418]
